<a href="https://colab.research.google.com/github/anon/ILCiteR/blob/main/cite_reco_s2orc_gather_meta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Course of action:
# Gather papers from the metadata parses
# Match papers in the PDF parses using their 'paper_id'

import json
import nltk

In [2]:
%%capture
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
location = 'drive/My Drive/cite_reco_s2orc/'

In [4]:
topic_names = ['summarization', 'machine translation', 'named entity recognition', 'sentiment analysis']
topic_abbs = ['summ', 'mt', 'ner', 'sa']

In [5]:
out_dir = 'out/'
look_for_file = 'found_paper_IDs.txt'
topic_papers = [[], [], [], []]
look_for_list = []

def add_to_look_for_file(paper_ID):
  global location, look_for_file
  with open(location + out_dir + look_for_file, 'a+') as f:
    f.write(str(paper_ID) + '\n')
  return

def add_to_topic_metas(meta_record, topic_id):
  global topic_abbs
  with open(location + out_dir + topic_abbs[topic_id] + '.jsonl', 'a+') as f:
    f.write(str(meta_record) + '\n')
  return

In [6]:
def normalize_text(text):
  tokens = nltk.word_tokenize(text.lower())
  return ' '.join(tokens).strip()

def merge_dicts(a, b):
    merged = {**a, **b}
    return merged

def process_paper(paper_meta):
  global topic_names, topic_papers, look_for_list
  if not paper_meta['has_pdf_parse']:
    return False

  if not paper_meta['abstract']:
    return False

  abstract_text = normalize_text(paper_meta['abstract'])
  outcome = False

  for index, topic in enumerate(topic_names):
    if topic in abstract_text:
      print('Topic:', topic)
      print('Title:', paper_meta['title'])
      print('Normalized Abstract:', abstract_text)
      print()

      topic_papers[index].append(paper_meta)
      add_to_topic_metas(paper_meta, index)

      look_for_list.append(paper_meta['paper_id'])
      add_to_look_for_file(paper_meta['paper_id'])

      outcome = True
  return outcome

def is_CS_paper(paper_meta):
  if not paper_meta['mag_field_of_study']:
    return False
  if 'Computer Science' not in paper_meta['mag_field_of_study']:
    return False
  return True

In [7]:
metadata_file_prefix = 'metadata_'

line_count = 0
successes = 0
cs_papers = 0

limit = 100000

with open(location + metadata_file_prefix + '1.jsonl') as m:
  while line_count < limit:
    m_line = m.readline()
    if not m_line:
      break

    line_count += 1
    paper_meta = json.loads(m_line)

    if not is_CS_paper(paper_meta):
      continue

    cs_papers += 1

    if process_paper(paper_meta):
      successes += 1

print('Number of fetches:', successes)
print('Number of CS papers: ' + str(cs_papers) + '/' +str(line_count))

Topic: machine translation
Title: Towards conceptual generalization in the embedding space
Normalized Abstract: humans are able to conceive physical reality by jointly learning different facets thereof . to every pair of notions related to a perceived reality may correspond a mutual relation , which is a notion on its own , but one-level higher . thus , we may have a description of perceived reality on at least two levels and the translation map between them is in general , due to their different content corpus , one-to-many . following success of the unsupervised neural machine translation models , which are essentially one-to-one mappings trained separately on monolingual corpora , we examine further capabilities of the unsupervised deep learning methods used there and apply some of these methods to sets of notions of different level and measure . using the graph and word embedding-like techniques , we build one-to-many map without parallel data in order to establish a unified vector

In [8]:
# That's it